In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

from network.srgan_deploy import SRGAN

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
gan = SRGAN()
inputimg = "imageValidate/butterfly_GT.bmp"

In [3]:
# Load image & scale it
img_hr = cv2.imread(inputimg, cv2.IMREAD_COLOR)
img_hr = cv2.cvtColor(img_hr, cv2.COLOR_BGR2RGB)

img_hr = img_hr.astype(np.float) / 127.5 - 1

# Create a low-resolution version of it
#lr_shape = (int(img_hr.shape[1]/4), int(img_hr.shape[0]/4))
#img_lr = cv2.resize(img_hr, lr_shape, cv2.INTER_CUBIC)
img_lr = img_hr

# Predict high-resolution version (add batch dimension to image)
img_sr = gan.generator.predict(np.expand_dims(img_lr, 0))

# Remove batch dimension
img_sr = np.squeeze(img_sr, axis=0)

In [4]:
pltShow = False

if pltShow:
    # Images and titles
    images = {
        'Low Resolution': img_lr, 'SRGAN': img_sr, 'Original': img_hr
    }

    # Plot the images. Note: rescaling and using squeeze since we are getting batches of size 1                    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    for i, (title, img) in enumerate(images.items()):
        axes[i].imshow(0.5 * img + 0.5)
        axes[i].set_title(title)
        axes[i].axis('off')
    plt.show()

In [5]:
srout = 127.5 * img_sr + 127.5
srout[srout[:] > 255] = 255
srout[srout[:] < 0] = 0
srout = cv2.cvtColor(srout, cv2.COLOR_BGR2RGB)
cv2.imwrite('sr.png', srout, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])

hrout = cv2.imread(inputimg, cv2.IMREAD_COLOR)
lr_shape = (int(hrout.shape[1]*4), int(hrout.shape[0]*4))
lrout = cv2.resize(hrout, lr_shape, cv2.INTER_CUBIC)
cv2.imwrite('lr.png', lrout, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])

True

In [6]:
import tensorflowjs as tfjs

tfjsExport = True
FILEPATH_TFJS = "./tfjs"

if tfjsExport:
    tfjs.converters.save_keras_model(gan.generator, FILEPATH_TFJS)
    print(FILEPATH_TFJS + " save complete")

./tfjs save complete
